In [1]:
import json 
json_dir = '/root/notebooks/0858611-2/tooth_xray/model/700_700/mapping.json'
with open(json_dir) as json_file:
    json_data = json.load(json_file)

In [2]:
def get_level(mean):
    if mean>=0 and mean<=2:
        return 'normal'
    elif mean>=3 and mean<=4:
        return 'medium'
    elif mean>4:
        return 'serious'
    else:
        return 'over_range'

In [3]:
from statistics import mean
import pandas as pd
img_path = []
img_cal = []
img_cal_mean = []
img_cal_level = []

for key in json_data:
    try:
        new_path = key.replace("Dataset/Images", "/root/notebooks/0858611-2/tooth_xray/model/700_700/Images")
        max_cal = int(max(json_data[key]))
        img_cal_level.append(get_level(max_cal))
        img_cal_mean.append(str(max_cal))
        img_path.append(new_path)
        img_cal.append(json_data[key])
    except:
        #print(key,json_data[key])
        pass

In [4]:
print(len(img_path))
print(len(img_cal))
print(len(img_cal_mean))
print(len(img_cal_level))
print(img_cal_level.count('normal'))
print(img_cal_level.count('medium'))
print(img_cal_level.count('serious'))

1493
1493
1493
1493
802
456
223


In [5]:
import pandas as pd

dict = {"image_path": img_path,  
        "img_cal": img_cal,
        "img_cal_mean":img_cal_mean,
        "img_cal_level":img_cal_level
       }

tooth_dataframe = pd.DataFrame(dict)
print(tooth_dataframe.head())

                                          image_path img_cal img_cal_mean  \
0  /root/notebooks/0858611-2/tooth_xray/model/700...  [5, 3]            5   
1  /root/notebooks/0858611-2/tooth_xray/model/700...  [3, 4]            4   
2  /root/notebooks/0858611-2/tooth_xray/model/700...  [7, 9]            9   
3  /root/notebooks/0858611-2/tooth_xray/model/700...  [4, 5]            5   
4  /root/notebooks/0858611-2/tooth_xray/model/700...  [4, 3]            4   

  img_cal_level  
0       serious  
1        medium  
2       serious  
3       serious  
4        medium  


In [6]:
tooth_dataframe = tooth_dataframe[tooth_dataframe.img_cal_level != 'over_range']

normal_dataframe = tooth_dataframe[tooth_dataframe.img_cal_level == 'normal']
medium_dataframe = tooth_dataframe[tooth_dataframe.img_cal_level == 'medium']
serious_dataframe = tooth_dataframe[tooth_dataframe.img_cal_level == 'serious']

In [9]:
import numpy as np

def label_list_to_num(train_label):
    label_array = []
    for symptom in train_label:
        if symptom == 'normal':
            label_array.append(0)
        if symptom == 'medium':
            label_array.append(1)
        if symptom == 'serious':
            label_array.append(2)
    return np.array(label_array)

data_path_list = tooth_dataframe.image_path
data_label_array = label_list_to_num(tooth_dataframe.img_cal_level)

all_img_path = data_path_list
all_img_label = data_label_array
print(len(all_img_path))
print(len(all_img_label))

1481
1481


In [32]:
#create tain data
train_data_path = all_img_path[:1200] 
train_data_label = all_img_label[:1200]


img_path_rotate = train_data_path
image_label_array = train_data_label
nums = 24000
save_rotate_img_dir = '/root/notebooks/0858611-2/tooth_xray/model/700_700/image_generate_all_rotate/train'
save_csv_path = '/root/notebooks/0858611-2/tooth_xray/model/700_700/image_generate_all_rotate/train_label.csv'

In [39]:
#create test data
test_data_path = all_img_path[1200:] 
test_data_label = all_img_label[1200:]


img_path_rotate = test_data_path
image_label_array = test_data_label
nums = 5620
save_rotate_img_dir = '/root/notebooks/0858611-2/tooth_xray/model/700_700/image_generate_all_rotate/test'
save_csv_path = '/root/notebooks/0858611-2/tooth_xray/model/700_700/image_generate_all_rotate/test_label.csv'

In [40]:
import PIL, os
from PIL import Image
import random 
import csv

# img_path_rotate = img_path[:5] 
# # It is your img_path list  

# image_label_array = train_label_array[:5] 
# # It is label of above img_path

# nums = 5
# # It should small than imgs*degree range  
# # ex:get 3 img and degree_range is [-5,5] ,should small than 3*(5-(-5)) = 30 

# save_rotate_img_dir = '/root/notebooks/0858611-2/tooth_xray/model/700_700/image_generate_all_rotate'
# #save_img_path
# save_csv_path = '/root/notebooks/0858611-2/tooth_xray/model/700_700/image_generate_all_rotate/train_label.csv'
# #save_csv_path it contain rotate_img_path & label


ratate_range = [-10,10] 
# means -10 drgree to 10 degree

def rotate_save_img(origin_img,rotate_image_name,label,save_dir,save_csv_path,degree,count):
    img = Image.open(origin_img)
    img.rotate(degree).save(rotate_image_name)
    with open(save_csv_path, 'a') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([rotate_image_name,label])
    count += 1
    return count

def create_ratate_images(image_path_list,image_label_array,nums,save_dir,save_csv_path,ratate_range=[0,1]):
    if len(image_path_list)*(ratate_range[1]-ratate_range[0]) < nums:
        print('You have too few image with all rotate degree')
        print('nums max is {}'.format(len(image_path_list)*(ratate_range[1]-ratate_range[0])))
        return
    ratate_img_path_list = []
    rotate_image_label_array =[]
    count = 0
    while count < nums:
        for origin_img,label in zip(image_path_list,image_label_array):
            if count < nums:
                degree = random.randrange(ratate_range[0],ratate_range[1])
                result_name = origin_img.split('/')[-1][:-4]
                rotate_image_name = save_dir+'/'+result_name+'_'+'rotate'+'_'+str(degree)+ '.png'
                while os.path.exists(rotate_image_name):
                    degree = random.randrange(ratate_range[0],ratate_range[1])
                    rotate_image_name = save_dir+'/'+result_name+'_'+'rotate'+'_'+str(degree)+ '.png'
                count = rotate_save_img(origin_img,rotate_image_name,label,save_dir,save_csv_path,degree,count)
            else:
                return ratate_img_path_list,rotate_image_label_array
    return ratate_img_path_list,rotate_image_label_array

if os.path.exists(save_csv_path) == True:
    print('csv is exist')
else:
    with open(save_csv_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Image_path','label'])
        
create_ratate_images(img_path_rotate,image_label_array,nums,save_rotate_img_dir,save_csv_path,[-10,10])

csv is exist


([], [])